In [1]:
#Imports
from src.api_calls import get_match_history, get_match_details
from src.cleaners import id_list_from_history, clean_match_details, make_csv
from src.populate_db import populate_db
import requests
import pymongo
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.dota_data
from src.heroes import heroes, name_id, id_name
from src.eda import make_counter
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from src.models import train_rfc

In [3]:
#data = db.raw.find()
c = make_counter(db.raw.find())
make_csv(c, db.raw.find())

In [4]:
df = pd.read_csv('test.csv')

In [5]:
df.head()

,Unnamed: 0,match_id,match_date,radiant_win,1R,1D,2R,2D,3R,3D,...,120R,120D,121R,121D,126R,126D,128R,128D,129R,129D
0,0,5452517913,1591222263,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,5452438801,1591217166,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2,5452379337,1591213905,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,5452166452,1591204222,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,5452093536,1591201406,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [6]:
train_rfc(df)

In [12]:
drop_cols = ['Unnamed: 0', 'match_id', 'match_date', 'Unnamed: 1']
for i in drop_cols:
    try:
        df.pop(i)
    except:
        continue

In [13]:
df.head()

,radiant_win,1R,1D,2R,2D,3R,3D,4R,4D,5R,...,120R,120D,121R,121D,126R,126D,128R,128D,129R,129D
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [14]:
trf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [15]:
y = df.pop('radiant_win')
X = df
trf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
# from collections import Counter
# c = Counter()
# for match in data:
#     c['count'] += 1
#     if c['count'] % 100 == 0:
#         print(c['count'])
#     try:
#         for indx, hero_id in enumerate(match['radiant_hero_ids']):
#             c[hero_id] += 1
#             c[match['dire_hero_ids'][indx]] += 1
#     except:
#         print(match)


In [21]:
# match = clean_match_details(get_match_details(5140440454))
# match

{'_id': 5140440454,
 'radiant_win': 1,
 'match_date': 1575441650,
 'radiant_hero_ids': [111, 97, 13, 104, 70],
 'dire_hero_ids': [87, 41, 9, 75, 98]}

In [22]:
# db.raw.update_one({'_id':5140440454}, {"$set": match}, upsert=True)

In [23]:
# for indx, hid in enumerate(match['radiant_hero_ids']):
#     c[hid] += 1
#     c[match['dire_hero_ids'][indx]] += 1

In [ ]:
# import sys
# #start
# #for runs
# match_IDs = None
# for i in range(3):
#     print(f'Run: {i+1}')
#     #call pro matches api - if list of match id's use lowest match id as last id
#     #get list of match id's
#     if match_IDs == None:
#         match_IDs = id_list_from_history(get_match_history())
#     else:
#         match_IDs = id_list_from_history(get_match_history(match_IDs[-1]))  
#     #for each id
#     for match_id in match_IDs[:5]:
#         try:
#             #insert just match id to check for duplicates
#             db.raw.insert_one({'_id': match_id})
#             #call & clean match detail api
#             match = clean_match_details(get_match_details(match_id))
#             #add data to db
#             db.raw.update_one({'_id':match['_id']}, {"$set": match}, True)
#             #print(f'Match {match_id} inserted')
#         except pymongo.errors.DuplicateKeyError:
#             print('Match ID was already in the DB')
#         except:
#             print(f'Unknown Error: {sys.exc_info()[0]}')

In [2]:
# #call pro matches api - if list of match id's use lowest match id as last
# #get list of match id's
# check = []
# match_IDs = id_list_from_history(get_match_history())
# for match_id in match_IDs:
#     match = clean_match_details(get_match_details(match_id))
#     check += [match]
# db.raw.insert_many(check)
# #for each id
#     #if id not in db
#         #call match detail api
#         #clean and add data to db

In [4]:
# del test['count']
# uids = sorted(test.keys())
# uid_cols =  [] #[[(str(i)+'R'),(str(i)+'D')] for i in uids]
# for i in uids:
#     uid_cols += [(str(i)+'R')]
#     uid_cols += [(str(i)+'D')]
# #uid_cols
# columns = ['match_id', 'match_date', 'radiant_win']
# columns += uid_cols
# #columns

In [11]:
# row_template = {col: 0 for col in columns}
# rows_list = []
# for match in db.raw.find():
#     temp_row = row_template.copy()
#     temp_row['match_id'] = match['_id']
#     temp_row['match_date'] = match['match_date']
#     temp_row['radiant_win'] = match['radiant_win']
#     for indx, hid in enumerate(match['radiant_hero_ids']):
#         temp_row[(str(hid)+'R')] = 1
#         temp_row[(str(match['dire_hero_ids'][indx])+'D')] = 1
#     rows_list += [temp_row]

In [13]:
# #https://stackoverflow.com/questions/10715965/add-one-row-to-pandas-dataframe/17496530#17496530
# import pandas as pd
# df = pd.DataFrame(rows_list)
# df.to_csv('test.csv')

In [ ]:
# {'_id': 5140440454,
#  'radiant_win': 1,
#  'match_date': 1575441650,
#  'radiant_hero_ids': [111, 97, 13, 104, 70],
#  'dire_hero_ids': [87, 41, 9, 75, 98]}

In [9]:
x = [heroes[str(hero)]['localized_name'] for hero in her

['Anti-Mage',
 'Axe',
 'Bane',
 'Bloodseeker',
 'Crystal Maiden',
 'Drow Ranger',
 'Earthshaker',
 'Juggernaut',
 'Mirana',
 'Morphling',
 'Shadow Fiend',
 'Phantom Lancer',
 'Puck',
 'Pudge',
 'Razor',
 'Sand King',
 'Storm Spirit',
 'Sven',
 'Tiny',
 'Vengeful Spirit',
 'Windranger',
 'Zeus',
 'Kunkka',
 'Lina',
 'Lion',
 'Shadow Shaman',
 'Slardar',
 'Tidehunter',
 'Witch Doctor',
 'Lich',
 'Riki',
 'Enigma',
 'Tinker',
 'Sniper',
 'Necrophos',
 'Warlock',
 'Beastmaster',
 'Queen of Pain',
 'Venomancer',
 'Faceless Void',
 'Wraith King',
 'Death Prophet',
 'Phantom Assassin',
 'Pugna',
 'Templar Assassin',
 'Viper',
 'Luna',
 'Dragon Knight',
 'Dazzle',
 'Clockwerk',
 'Leshrac',
 "Nature's Prophet",
 'Lifestealer',
 'Dark Seer',
 'Clinkz',
 'Omniknight',
 'Enchantress',
 'Huskar',
 'Night Stalker',
 'Broodmother',
 'Bounty Hunter',
 'Weaver',
 'Jakiro',
 'Batrider',
 'Chen',
 'Spectre',
 'Ancient Apparition',
 'Doom',
 'Ursa',
 'Spirit Breaker',
 'Gyrocopter',
 'Alchemist',
 'Invoke